<a href="https://colab.research.google.com/github/alireza-siami/Python_Projects/blob/DATAQUEST's-Projects/Profitable_App_Profiles_for_the_App_Store_and_Google_Play_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Profitable App Profiles for the App Store and Google Play Market**

In this project, I will follow the guided project by DATAQUEST as 'Profitable App Profiles for the App Store and Google Play Markets'. Its goal is to determine which trends and genres are popular among mobile App users. For that, two datasets provided by Kaggle for Android users and IOS users will be explored.

In [44]:
# import the necessary library
from csv import reader

In [45]:
# load Applestore data to a list
apple_file = open('AppleStore.csv')
apple_csv = reader(apple_file)
apple_list = list(apple_csv)
apple_columns = apple_list[0]
apple = apple_list[1:]

# load googleplaystore data to a list
google_file = open('googleplaystore.csv')
google_csv = reader(google_file)
google_list = list(google_csv)
google_columns = google_list[0]
google = google_list[1:]

##**Define a function to explore datasets**

In [3]:
def explore_data(dataset,start,end,rows_and_columns=False):
  data = dataset[start:end]
  for row in data:
    print(row,'\n')

  if rows_and_columns:
    print(f'Number of rows: {len(dataset)}\n')
    print(f'Number of columns: {len(data[0])}\n')

In [46]:
# check some data in data sets
explore_data(apple,0,2,True)
print(apple_columns,'\n')

explore_data(google,0,2,True)
print(google_columns,'\n')

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

Number of rows: 7197

Number of columns: 16

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

Number of rows: 10841

Number of columns: 13

['App', 'Category', 'Rating

##**Delete the row with NaN in Google dataset**

In [47]:
# deleting NaN row according to https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/66015
print(google[10472])
del(google[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [6]:
print(google[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


##**Explore the duplicate Apps in Google dataset**

In [7]:
google_duplicate = []
google_unique = []

for row in google:
  if row[0] not in google_unique:
    google_unique.append(row[0])
  else:
    google_duplicate.append(row[0])

print(len(google_unique), google_unique[:5])
print(len(google_duplicate), google_duplicate[:5])

9659 ['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'Sketch - Draw & Paint', 'Pixel Draw - Number Art Coloring Book']
1181 ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


By exploring the differences between duplicate Apps we notice that they are almost the same and the main difference is in column 4 which corresponds to the number of reviews. This is because the data have been gathered at different times so the number of reviews of the App has changed. Therefore, the better option to delete duplicate rows is to delete duplicate rows with fewer reviews. Hence, we will have the most recent reviews of that App.  

In [48]:
# examin Instagram in Google dataset to see the diffrences among duplicate Apps
print(google_columns,'\n')

for row in google:
  if row[0]=='Instagram':
    print(row,'\n')

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 

['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 

['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device'] 



Here we are going to delete duplicate Apps according to the above information.

In [49]:
google_with_high_reviews = {} # creating a dictionary to save high number of reviews for Apps

for row in google:

  if row[0] not in google_with_high_reviews:
    google_with_high_reviews[row[0]] = int(row[3])

  elif int(row[3]) > google_with_high_reviews[row[0]]:
    google_with_high_reviews[row[0]] = int(row[3])

print(google_with_high_reviews['Instagram'])

66577446


In [50]:
google_modified = []
google_apps = [] # creating a list of Apps'name that we are going to store them in google_modified list

for row in google:

  if row[0] not in google_apps and google_with_high_reviews[row[0]] == int(row[3]):
    google_apps.append(row[0])
    google_modified.append(row)

len(google_modified)

9659

In [11]:
len(google) - len(google_duplicate)

9659

##**Explore the duplicate Apps in Apple dataset**

There is not any duplicate Apps in Apple dataset.

In [12]:
apple_duplicate = []
apple_unique = []

for row in apple:

  if row[0] not in apple_unique:
    apple_unique.append(row[0])

  else:
    apple_duplicate.append(row[0])

print(len(apple_duplicate), apple_duplicate)
print(len(apple_unique))
print(len(apple))

0 []
7197
7197


##**Explore the both datasets to delete non-english Apps**

As there are other language Apps in both datasets, we define a function to detect these Apps from datasets.

In [13]:
def FindNonEnglishApps(datasets, name_column = 0):
  NonEnglish = []

  for row in datasets:
    b = 0

    for s in row[name_column]:

      if ord(s) > 127:
        b += 1

        if b > 3:
          NonEnglish.append(row[name_column])
          break

  return(NonEnglish)

In [14]:
google_non_english = FindNonEnglishApps(google_modified)
len(google_non_english)

45

In [15]:
apple_non_english = FindNonEnglishApps(apple,1)
len(apple_non_english)

1014

In [16]:
google_modified_english = []
apple_english = []

for row in google_modified:

  if row[0] not in google_non_english:
    google_modified_english.append(row)

for row in apple:

  if row[1] not in apple_non_english:
    apple_english.append(row)

explore_data(google_modified_english,0,2,True)
explore_data(apple_english,0,2,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Number of rows: 9614

Number of columns: 13

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

Number of rows: 6183

Number of columns: 16



##**Exploring datasets to identify free Apps**
Here we are going to find free Apps. We should consider that price's format in datasets is string.

In [51]:
# defining a function to identify free Apps

def FindFreeApps(dataset= google_modified_english, price_column= 7):
  free_apps = []

  for row in dataset:
    if row[price_column] == '0' or row[price_column] == '0.0':
      free_apps.append(row)
  return(free_apps)

In [18]:
google_modified_english_free = FindFreeApps()
apple_english_free = FindFreeApps(apple_english, 4)

explore_data(google_modified_english_free,0,2,True)
explore_data(apple_english_free,0,2,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Number of rows: 8864

Number of columns: 13

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

Number of rows: 3222

Number of columns: 16



##**Most common Apps by genre**
Here we are going to identify the common Apps in both datasets. Since our final goal is to create an App that would be profitable in both the Google and Apple stores, we should first be certain which genres are most popular in both Apple and Android users. First, we need to explore the column names to identify which columns will tell us about genres.

In [19]:
print(google_columns)
explore_data(google_modified_english_free, 0, 1)
print(apple_columns)
explore_data(apple_english_free,0,1)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 



According to the above, we can see that the columns with 'Genres' and 'Category' titles in Google dataset and with 'prime_genre' in Apple dataset are the ones we can use to identify popular genres.




In [20]:
def frequency_table(dataset= google_modified_english_free, genre_column= 1):
  frequency = {}
  percentage = {}
  t = 1

  for row in dataset:
    t += 1

    if row[genre_column] in frequency:
      frequency[row[genre_column]] += 1
    else:
      frequency[row[genre_column]] = 1

  for key in frequency:
    percentage[key] = frequency[key]/t*100

  return(percentage)

In [21]:
google_frequency = frequency_table(genre_column=1)
print(google_frequency)

apple_frequency = frequency_table(apple_english_free, 11)
print(apple_frequency)

{'ART_AND_DESIGN': 0.6429780033840948, 'AUTO_AND_VEHICLES': 0.924985899605189, 'BEAUTY': 0.5978567399887197, 'BOOKS_AND_REFERENCE': 2.143260011280316, 'BUSINESS': 4.591088550479413, 'COMICS': 0.6204173716864072, 'COMMUNICATION': 3.2374506486181613, 'DATING': 1.8612521150592216, 'EDUCATION': 1.161872532430908, 'ENTERTAINMENT': 0.9588268471517203, 'EVENTS': 0.7106598984771574, 'FINANCE': 3.699943598420756, 'FOOD_AND_DRINK': 1.2408347433728144, 'HEALTH_AND_FITNESS': 3.0795262267343486, 'HOUSE_AND_HOME': 0.8234630569655951, 'LIBRARIES_AND_DEMO': 0.9362662154540328, 'LIFESTYLE': 3.902989283699944, 'GAME': 9.723632261703328, 'FAMILY': 18.905809362662154, 'MEDICAL': 3.5307388606880994, 'SOCIAL': 2.662154540327129, 'SHOPPING': 2.2447828539199097, 'PHOTOGRAPHY': 2.9441624365482233, 'SPORTS': 3.395375070501974, 'TRAVEL_AND_LOCAL': 2.33502538071066, 'TOOLS': 8.460236886632826, 'PERSONALIZATION': 3.3164128595600673, 'PRODUCTIVITY': 3.8917089678511, 'PARENTING': 0.6542583192329385, 'WEATHER': 0.800

Now, we are going to sort the percentage dictionary from high to low to get the sorted genres for each dataset. For that, we first define a function.

In [22]:
def sort_genres(dataset, genre_column, reverse=True):
  percentage = frequency_table(dataset,genre_column)
  sorted_dict = {k:v for k,v in sorted(percentage.items(), key= lambda item:item[1], reverse= reverse)}

  for k,v in sorted_dict.items():
    print(f'{k} : {v}')

Now, we have a function that gets the dataset and the index of the column as input, and then by using another function, it calculates the percentage frequency of that column and sorts it. We chose the 'Category' and 'Genres' columns for the Google dataset and the 'prime_genre' column for the Apple dataset. Their sorted results can be found below:

In [23]:
sort_genres(google_modified_english_free,1) # Sort genres by 'Category' for google datset

FAMILY : 18.905809362662154
GAME : 9.723632261703328
TOOLS : 8.460236886632826
BUSINESS : 4.591088550479413
LIFESTYLE : 3.902989283699944
PRODUCTIVITY : 3.8917089678511
FINANCE : 3.699943598420756
MEDICAL : 3.5307388606880994
SPORTS : 3.395375070501974
PERSONALIZATION : 3.3164128595600673
COMMUNICATION : 3.2374506486181613
HEALTH_AND_FITNESS : 3.0795262267343486
PHOTOGRAPHY : 2.9441624365482233
NEWS_AND_MAGAZINES : 2.7975183305132543
SOCIAL : 2.662154540327129
TRAVEL_AND_LOCAL : 2.33502538071066
SHOPPING : 2.2447828539199097
BOOKS_AND_REFERENCE : 2.143260011280316
DATING : 1.8612521150592216
VIDEO_PLAYERS : 1.793570219966159
MAPS_AND_NAVIGATION : 1.3987591652566271
FOOD_AND_DRINK : 1.2408347433728144
EDUCATION : 1.161872532430908
ENTERTAINMENT : 0.9588268471517203
LIBRARIES_AND_DEMO : 0.9362662154540328
AUTO_AND_VEHICLES : 0.924985899605189
HOUSE_AND_HOME : 0.8234630569655951
WEATHER : 0.8009024252679076
EVENTS : 0.7106598984771574
PARENTING : 0.6542583192329385
ART_AND_DESIGN : 0.6429

In [24]:
sort_genres(google_modified_english_free,9) # Sort genres by 'Genres' for google datset

Tools : 8.44895657078398
Entertainment : 6.068809926677947
Education : 5.346869712351946
Business : 4.591088550479413
Lifestyle : 3.8917089678511
Productivity : 3.8917089678511
Finance : 3.699943598420756
Medical : 3.5307388606880994
Sports : 3.4630569655950363
Personalization : 3.3164128595600673
Communication : 3.2374506486181613
Action : 3.102086858432036
Health & Fitness : 3.0795262267343486
Photography : 2.9441624365482233
News & Magazines : 2.7975183305132543
Social : 2.662154540327129
Travel & Local : 2.323745064861816
Shopping : 2.2447828539199097
Books & Reference : 2.143260011280316
Simulation : 2.0417371686407217
Dating : 1.8612521150592216
Arcade : 1.849971799210378
Video Players & Editors : 1.7710095882684715
Casual : 1.7597292724196276
Maps & Navigation : 1.3987591652566271
Food & Drink : 1.2408347433728144
Puzzle : 1.1280315848843767
Racing : 0.9926677946982515
Libraries & Demo : 0.9362662154540328
Role Playing : 0.9362662154540328
Auto & Vehicles : 0.924985899605189
Str

The differences between 'Category' and 'Genres' are not clear enough. But what we can see from these two is that the applicable Apps are more popular among Google users. While games and entertainment are the second most popular in both columns, the applicable Apps are among the 5 most popular Apps.

In [52]:
sort_genres(apple_english_free,11) # sort genres by 'prime_genre' for apple datset

Games : 58.144585789636984
Entertainment : 7.880856345020168
Photo & Video : 4.964318957493019
Education : 3.661185231151101
Social Networking : 3.288861309339125
Shopping : 2.6062674526838348
Utilities : 2.513186472230841
Sports : 2.140862550418864
Music : 2.04778156996587
Health & Fitness : 2.016754576481539
Productivity : 1.7375116351225566
Lifestyle : 1.5823766677008997
News : 1.3341607198262488
Travel : 1.2410797393732547
Finance : 1.1169717654359292
Weather : 0.8687558175612783
Food & Drink : 0.8067018305926157
Reference : 0.5584858827179646
Business : 0.5274588892336333
Book : 0.43437790878063914
Navigation : 0.18616196090598822
Medical : 0.18616196090598822
Catalogs : 0.12410797393732546


From the Apple dataset result, we can notice that the main popular genres are 'Games', 'Entertainment', and 'Photo & Video' which shows that the main trend among Apple users is the entertainment Apps.

##**Most popular Apps by genre in apple dataset**
We want to find which genre is the most popular according to the number of users. For that, we can use the 'rating_count_tot' column as an indicator for Apple users. By summing this column for all the apps belonging to one genre and then dividing them by the number of that genre's apps, we can find the average the 'rating_count_tot' for each genre and so we can find the most popular genres based on the number of users.

In [26]:
# Find the average number of users for each gener
genre_rate_dict = {}

for genre in apple_frequency:
  genre_rate = 0
  total_apps = 0

  for row in apple_english_free:

    if row[11] == genre:
      genre_rate += int(row[5])
      total_apps += 1
      genre_rate_dict[genre] = genre_rate/total_apps

sorted_genre_rate_dict = {k:v for k,v in sorted(genre_rate_dict.items(), key= lambda item:item[1],reverse = True)} # Sort the popular genres
sorted_genre_rate_dict

{'Navigation': 86090.33333333333,
 'Reference': 74942.11111111111,
 'Social Networking': 71548.34905660378,
 'Music': 57326.530303030304,
 'Weather': 52279.892857142855,
 'Book': 39758.5,
 'Food & Drink': 33333.92307692308,
 'Finance': 31467.944444444445,
 'Photo & Video': 28441.54375,
 'Travel': 28243.8,
 'Shopping': 26919.690476190477,
 'Health & Fitness': 23298.015384615384,
 'Sports': 23008.898550724636,
 'Games': 22788.6696905016,
 'News': 21248.023255813954,
 'Productivity': 21028.410714285714,
 'Utilities': 18684.456790123455,
 'Lifestyle': 16485.764705882353,
 'Entertainment': 14029.830708661417,
 'Business': 7491.117647058823,
 'Education': 7003.983050847458,
 'Catalogs': 4004.0,
 'Medical': 612.0}

We can see the most popular genres by number of users are 'Navigation', 'Reference' and 'Social Networking'. Now we explore this genres' Apps and their corresponding 'rating_count_tot'.

In [37]:
# define a function to return sorted each genres' Apps by their 'rating_count_tot'

def sorted_genre_apps_users(genre, dataset=apple_english_free,app_column=1, genre_columns=11,user_number_column=5):
  genre_apps_users = {}

  for row in dataset:
    if row[genre_columns] == genre:
      genre_apps_users[row[app_column]] = float(row[user_number_column])
  sorted_genre_apps_users = {k:v for k,v in sorted(genre_apps_users.items(), key= lambda item:item[1], reverse= True)}
  return(sorted_genre_apps_users)

In [28]:
sorted_genre_apps_users(genre='Navigation')

{'Waze - GPS Navigation, Maps & Real-time Traffic': 345046.0,
 'Google Maps - Navigation & Transit': 154911.0,
 'Geocaching®': 12811.0,
 'CoPilot GPS – Car Navigation & Offline Maps': 3582.0,
 'ImmobilienScout24: Real Estate Search in Germany': 187.0,
 'Railway Route Search': 5.0}

In [29]:
sorted_genre_apps_users(genre='Reference')

{'Bible': 985920.0,
 'Dictionary.com Dictionary & Thesaurus': 200047.0,
 'Dictionary.com Dictionary & Thesaurus for iPad': 54175.0,
 'Google Translate': 26786.0,
 'Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran': 18418.0,
 'New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition': 17588.0,
 'Merriam-Webster Dictionary': 16849.0,
 'Night Sky': 12122.0,
 'City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE)': 8535.0,
 'LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools': 4693.0,
 'GUNS MODS for Minecraft PC Edition - Mods Tools': 1497.0,
 'Guides for Pokémon GO - Pokemon GO News and Cheats': 826.0,
 'WWDC': 762.0,
 'Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free': 718.0,
 'VPN Express': 14.0,
 'Real Bike Traffic Rider Virtual Reality Glasses': 8.0,
 '教えて!goo': 0.0,
 'Jishokun-Japanese English Dictionary & Translator': 0.0}

In [30]:
sorted_genre_apps_users(genre='Social Networking')

{'Facebook': 2974676.0,
 'Pinterest': 1061624.0,
 'Skype for iPhone': 373519.0,
 'Messenger': 351466.0,
 'Tumblr': 334293.0,
 'WhatsApp Messenger': 287589.0,
 'Kik': 260965.0,
 'ooVoo – Free Video Call, Text and Voice': 177501.0,
 'TextNow - Unlimited Text + Calls': 164963.0,
 'Viber Messenger – Text & Call': 164249.0,
 'Followers - Social Analytics For Instagram': 112778.0,
 'MeetMe - Chat and Meet New People': 97072.0,
 'We Heart It - Fashion, wallpapers, quotes, tattoos': 90414.0,
 'InsTrack for Instagram - Analytics Plus More': 85535.0,
 'Tango - Free Video Call, Voice and Chat': 75412.0,
 'LinkedIn': 71856.0,
 'Match™ - #1 Dating App.': 60659.0,
 'Skype for iPad': 60163.0,
 'POF - Best Dating App for Conversations': 52642.0,
 'Timehop': 49510.0,
 'Find My Family, Friends & iPhone - Life360 Locator': 43877.0,
 'Whisper - Share, Express, Meet': 39819.0,
 'Hangouts': 36404.0,
 'LINE PLAY - Your Avatar World': 34677.0,
 'WeChat': 34584.0,
 'Badoo - Meet New People, Chat, Socialize.': 

By exploring the results of these three genres we notice that they are mostly dominated by big companies and Apps. For instance, popularity based on 'rating_count_tot' for 'Social Networking' is dominated by 'Facebook', 'Pinterest', 'Skype for iPhone', 'Messenger', and 'Tumblr' Apps. 'Reference' top Apps show us that book-related Apps are popular and it provides the opportunity to take another book and turn it into an App with other options for that book like searching in a book or an audio version of that book.

##**Most popular Apps by genre in apple dataset**
Here we want to do the same thing for the Google database and determine the most popular Apps by the number of users. According to the columns title of the Google dataset, it seems that the 'Installs' column shows the number of users. First, we try to find out the average of 'Installs' of apps for each genre. We should consider that the format of installs is string and they are in 10,000+, 100,000+, and so on formats. So first we should change it to the proper format of float.

In [35]:
genre_instals_dict = {}

for genre in google_frequency:
  genre_installs = 0
  total_apps = 0

  for row in google_modified_english_free:
    if row[1] == genre:
      total_apps += 1
      n_installs = row[5].replace('+','') # delete '+' and ',' from the first fromt of Installs
      n_installs = float(n_installs.replace(',',''))
      genre_installs += n_installs
      genre_instals_dict[genre] = genre_installs/total_apps

sorted_gener_installs_dict = {k:v for k,v in sorted(genre_instals_dict.items(), key= lambda item:item[1], reverse= True)}
sorted_gener_installs_dict

{'COMMUNICATION': 38456119.167247385,
 'VIDEO_PLAYERS': 24727872.452830188,
 'SOCIAL': 23253652.127118643,
 'PHOTOGRAPHY': 17840110.40229885,
 'PRODUCTIVITY': 16787331.344927534,
 'GAME': 15588015.603248259,
 'TRAVEL_AND_LOCAL': 13984077.710144928,
 'ENTERTAINMENT': 11640705.88235294,
 'TOOLS': 10801391.298666667,
 'NEWS_AND_MAGAZINES': 9549178.467741935,
 'BOOKS_AND_REFERENCE': 8767811.894736841,
 'SHOPPING': 7036877.311557789,
 'PERSONALIZATION': 5201482.6122448975,
 'WEATHER': 5074486.197183099,
 'HEALTH_AND_FITNESS': 4188821.9853479853,
 'MAPS_AND_NAVIGATION': 4056941.7741935486,
 'FAMILY': 3695641.8198090694,
 'SPORTS': 3638640.1428571427,
 'ART_AND_DESIGN': 1986335.0877192982,
 'FOOD_AND_DRINK': 1924897.7363636363,
 'EDUCATION': 1833495.145631068,
 'BUSINESS': 1712290.1474201474,
 'LIFESTYLE': 1437816.2687861272,
 'FINANCE': 1387692.475609756,
 'HOUSE_AND_HOME': 1331540.5616438356,
 'DATING': 854028.8303030303,
 'COMICS': 817657.2727272727,
 'AUTO_AND_VEHICLES': 647317.8170731707

From above it is clear that 'COMMUNICATION', 'VIDEO_PLAYERS', and 'SOCIAL' genres have more installs. We explore their apps below. Since we explore the 'Reference" genre for the Apple dataset, We will explore book-related genres for the Google dataset as well which is the 'BOOKS_AND_REFERENCE' genre.

In [39]:
def sorted_google_genre_apps_installs(genre):
  genre_apps_installs = {}

  for row in google_modified_english_free:
    if row[1] == genre:
      n_installs = row[5].replace('+','')
      n_installs = float(n_installs.replace(',',''))
      genre_apps_installs[row[0]] = n_installs

  return({k:v for k,v in sorted(genre_apps_installs.items(), key= lambda item:item[1], reverse=True)})

In [40]:
sorted_google_genre_apps_installs('COMMUNICATION')

{'WhatsApp Messenger': 1000000000.0,
 'Messenger – Text and Video Chat for Free': 1000000000.0,
 'Skype - free IM & video calls': 1000000000.0,
 'Google Chrome: Fast & Secure': 1000000000.0,
 'Gmail': 1000000000.0,
 'Hangouts': 1000000000.0,
 'Google Duo - High Quality Video Calls': 500000000.0,
 'imo free video calls and chat': 500000000.0,
 'LINE: Free Calls & Messages': 500000000.0,
 'UC Browser - Fast Download Private & Secure': 500000000.0,
 'Viber Messenger': 500000000.0,
 'imo beta free calls and text': 100000000.0,
 'Android Messages': 100000000.0,
 'Who': 100000000.0,
 'GO SMS Pro - Messenger, Free Themes, Emoji': 100000000.0,
 'Firefox Browser fast & private': 100000000.0,
 'Messenger Lite: Free Calls & Messages': 100000000.0,
 'Kik': 100000000.0,
 'KakaoTalk: Free Calls & Text': 100000000.0,
 'Opera Mini - fast web browser': 100000000.0,
 'Opera Browser: Fast and Secure': 100000000.0,
 'Telegram': 100000000.0,
 'Truecaller: Caller ID, SMS spam blocking & Dialer': 100000000.0

In [41]:
sorted_google_genre_apps_installs('VIDEO_PLAYERS')

{'YouTube': 1000000000.0,
 'Google Play Movies & TV': 1000000000.0,
 'MX Player': 500000000.0,
 'Motorola Gallery': 100000000.0,
 'VLC for Android': 100000000.0,
 'Dubsmash': 100000000.0,
 'VivaVideo - Video Editor & Photo Movie': 100000000.0,
 'VideoShow-Video Editor, Video Maker, Beauty Camera': 100000000.0,
 'Motorola FM Radio': 100000000.0,
 'Vote for': 50000000.0,
 'Vigo Video': 50000000.0,
 'MiniMovie - Free Video and Slideshow Editor': 50000000.0,
 'Samsung Video Library': 50000000.0,
 'LIKE – Magic Video Maker & Community': 50000000.0,
 'DU Recorder – Screen Recorder, Video Editor, Live': 50000000.0,
 'KineMaster – Pro Video Editor': 50000000.0,
 'VMate': 50000000.0,
 'HD Video Downloader : 2018 Best video mate': 50000000.0,
 'Ringdroid': 50000000.0,
 'Video Downloader': 10000000.0,
 'Video Player All Format': 10000000.0,
 'Code': 10000000.0,
 'Music - Mp3 Player': 10000000.0,
 'YouTube Studio': 10000000.0,
 'video player for android': 10000000.0,
 'HTC Service － DLNA': 1000000

In [42]:
sorted_google_genre_apps_installs('SOCIAL')

{'Facebook': 1000000000.0,
 'Google+': 1000000000.0,
 'Instagram': 1000000000.0,
 'Facebook Lite': 500000000.0,
 'Snapchat': 500000000.0,
 'Tumblr': 100000000.0,
 'Pinterest': 100000000.0,
 'Badoo - Free Chat & Dating App': 100000000.0,
 'Tango - Live Video Broadcast': 100000000.0,
 'LinkedIn': 100000000.0,
 'Tik Tok - including musical.ly': 100000000.0,
 'BIGO LIVE - Live Stream': 100000000.0,
 'VK': 100000000.0,
 'ooVoo Video Calls, Messaging & Stories': 50000000.0,
 'MeetMe: Chat & Meet New People': 50000000.0,
 'Zello PTT Walkie Talkie': 50000000.0,
 'POF Free Dating App': 50000000.0,
 'SKOUT - Meet, Chat, Go Live': 50000000.0,
 'TextNow - free text + calls': 10000000.0,
 'LiveMe - Video chat, new friends, and make money': 10000000.0,
 'HTC Social Plugin - Facebook': 10000000.0,
 'Quora': 10000000.0,
 'Kate Mobile for VK': 10000000.0,
 'Text Me: Text Free, Call Free, Second Phone Number': 10000000.0,
 'Text free - Free Text + Call': 10000000.0,
 'YouNow: Live Stream Video Chat': 10

In [43]:
sorted_google_genre_apps_installs('BOOKS_AND_REFERENCE')

{'Google Play Books': 1000000000.0,
 'Bible': 100000000.0,
 'Amazon Kindle': 100000000.0,
 'Wattpad 📖 Free Books': 100000000.0,
 'Audiobooks from Audible': 100000000.0,
 'Wikipedia': 10000000.0,
 'Cool Reader': 10000000.0,
 'FBReader: Favorite Book Reader': 10000000.0,
 'HTC Help': 10000000.0,
 'Moon+ Reader': 10000000.0,
 'Aldiko Book Reader': 10000000.0,
 'Al-Quran (Free)': 10000000.0,
 'Al Quran Indonesia': 10000000.0,
 "Al'Quran Bahasa Indonesia": 10000000.0,
 'Quran for Android': 10000000.0,
 'Dictionary.com: Find Definitions for English Words': 10000000.0,
 'English Dictionary - Offline': 10000000.0,
 'NOOK: Read eBooks & Magazines': 10000000.0,
 'Dictionary': 10000000.0,
 'Spanish English Translator': 10000000.0,
 'Dictionary - Merriam-Webster': 10000000.0,
 'JW Library': 10000000.0,
 'Oxford Dictionary of English : Free': 10000000.0,
 'English Hindi Dictionary': 10000000.0,
 'AlReader -any text book reader': 5000000.0,
 'Ebook Reader': 5000000.0,
 'Read books online': 5000000.0

According to the above information, we can see the same pattern in the Apple dataset for popular genres. Giant companies and Apps dominate the number of installs and they skew the average data. However, in the 'BOOKS_AND_REFERENCE' genre we can see the number of installs is less skewed and it shows us the opportunity for introducing a new App. We can get new ideas about new Apps by exploring the Apps in this genre and see how the main trend of Apps is in this genre.

##**Conclusion**
In this project, we tried to analyze data about the App Store and Google Play mobile apps to recommend an app profile that can be profitable for both markets.
It seems that taking a popular book and turning it into an app could be profitable for both the Google Play and the App Store markets. We can add some other features to this App. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

##**Refrences**
Guided project page:

https://www.dataquest.io/projects/guided-project-a-profitable-app-profiles-for-the-app-store-and-google-play-markets-2/

Datasets:

1- Google Play Store Apps:
https://www.kaggle.com/datasets/lava18/google-play-store-apps

2- IOS App Store:
https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps